In [1]:
import pandas as pd
import os
import numpy as np
import random
import joblib

DATA_DIR = '/mnt/sda/hong01-data/MART_DATA/OUTPUT_MERGED'
CSV_DIR = f"{DATA_DIR}/PANDAS"
IMG_DIR = f"{DATA_DIR}/AUTOGRAPHER"

In [2]:
print('===== DATA PREPARATION =====')
dfA = pd.read_csv(f'{CSV_DIR}/trainA.csv', index_col=0)
dfB = pd.read_csv(f'{CSV_DIR}/trainB.csv', index_col=0)
df = pd.concat([dfA, dfB])
df = df.sort_values(by=['event_id', 'sub_id'])

imgs = sorted(os.listdir(IMG_DIR))
imgs = [item[:5]+'test_'+item[5:] if 'pred' in item else item for item in imgs]

df_image = pd.DataFrame([item.split('_')+[item] for item in imgs], columns=['sub_id', 'source', 'event_id', 'img_order', 'image_path'])
df_image = df_image.astype({'sub_id': 'int64'})

df_train = pd.merge(df, df_image, how='inner', on=['sub_id', 'event_id', 'source'])
df_train['label'] = df_train.apply(lambda row: int(row['event_id'][-2:]), axis=1)

===== DATA PREPARATION =====


In [3]:
df['label'] = df.apply(lambda row: int(row['event_id'][-2:]), axis=1)
df

,sub_id,event_id,source,data_HR_activity_median,data_HR_activity_min,data_HR_activity_max,data_HR_activity_average,data_HR_activity_std,data_HR_activity_len,data_LEFT_ACC_MAG_median,...,data_AUTOGRAPHER_RESNET_max_coral fungus,data_AUTOGRAPHER_RESNET_max_agaric,data_AUTOGRAPHER_RESNET_max_gyromitra,"data_AUTOGRAPHER_RESNET_max_stinkhorn, carrion fungus",data_AUTOGRAPHER_RESNET_max_earthstar,"data_AUTOGRAPHER_RESNET_max_hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",data_AUTOGRAPHER_RESNET_max_bolete,"data_AUTOGRAPHER_RESNET_max_ear, spike, capitulum","data_AUTOGRAPHER_RESNET_max_toilet tissue, toilet paper, bathroom tissue",label
0,1001,act01,trainA,84.507042,76.923077,100.000000,84.577495,3.933112,90,0.000500,...,0.000090,0.000920,0.000048,0.000100,0.000287,0.000317,0.000290,0.001594,0.001767,1
0,1001,act01,trainB,85.714286,81.081081,92.307692,86.525331,2.709545,90,0.000850,...,0.000244,0.003576,0.000128,0.000239,0.000618,0.000757,0.000928,0.005534,0.009232,1
0,1002,act01,trainA,68.181818,60.606061,75.000000,67.920213,3.254272,90,-0.002704,...,0.000004,0.000071,0.000002,0.000003,0.000009,0.000100,0.000031,0.000402,0.000734,1
0,1002,act01,trainB,71.428571,67.415730,77.922078,71.630717,2.240689,90,-0.003506,...,0.000151,0.000966,0.000082,0.000151,0.000299,0.000926,0.000762,0.008168,0.005427,1
0,1003,act01,trainA,74.074074,66.666667,78.947368,73.875941,2.166754,90,0.021029,...,0.000295,0.007819,0.000449,0.000692,0.003246,0.004795,0.002219,0.007301,0.066273,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,1005,act20,trainB,98.360656,84.507042,105.263158,98.556521,3.391250,89,0.006852,...,0.000269,0.000881,0.000034,0.000167,0.000108,0.000597,0.000377,0.001633,0.750323,20
19,1006,act20,trainA,92.307692,85.714286,100.000000,91.705138,2.563889,90,0.003494,...,0.001381,0.001589,0.000075,0.000245,0.000297,0.001900,0.000491,0.003848,0.422092,20
19,1006,act20,trainB,93.750000,84.507042,103.448276,94.153427,4.012861,89,0.005112,...,0.001601,0.001030,0.000053,0.000363,0.000531,0.001294,0.000652,0.005349,0.296006,20
19,1007,act20,trainA,89.552239,74.074074,96.774194,87.873433,4.945947,89,0.021470,...,0.003216,0.001190,0.000040,0.000681,0.000378,0.000685,0.000530,0.010341,0.711271,20


In [4]:
df_train = df.copy()

In [5]:
data_columns = df_train.columns.str.startswith("data_")
data_autographer = [not item for item in df_train.columns.str.startswith("data_AUTOGRAPHER")]
cont_names = list(df_train.loc[:, data_columns & data_autographer].columns)
cont_names_2 = []
for col in cont_names:
    nunique = len(np.unique(df_train[col]))
    if nunique > 1:
        cont_names_2.append(col)
df_train_post = df_train[cont_names_2]
df_train_post = df_train_post.fillna(df_train_post.median())

In [6]:
standardize_paras = dict()
for col in cont_names_2:
    standardize_paras[col] = [df_train_post[col].mean(), df_train_post[col].std()]

In [7]:
standardize_paras

{'data_HR_activity_median': [82.80037262393816, 10.126603727472052],
 'data_HR_activity_min': [73.999984301409, 9.132487141642178],
 'data_HR_activity_max': [92.90461419618875, 10.231050877238824],
 'data_HR_activity_average': [82.9084797430595, 9.775180460212962],
 'data_HR_activity_std': [4.0375595638755275, 2.4973784839603863],
 'data_HR_activity_len': [89.71785714285714, 0.4508483214001913],
 'data_LEFT_ACC_MAG_median': [0.016747733146517077, 0.017667087700605412],
 'data_LEFT_ACC_MAG_min': [-0.5303886881543718, 0.22508111178559065],
 'data_LEFT_ACC_MAG_max': [1.3914852391193278, 0.39924991634548523],
 'data_LEFT_ACC_MAG_average': [0.023199174178477663, 0.025681894154484193],
 'data_LEFT_ACC_MAG_std': [0.07674702999991916, 0.07780738511878107],
 'data_LEFT_ACC_MAG_len': [7518.146428571428, 118.37303584194893],
 'data_LEFT_ACC_X_median': [-0.15716071428571426, 0.47061146744401644],
 'data_LEFT_ACC_X_min': [-1.0929642857142858, 0.6802061664138638],
 'data_LEFT_ACC_X_max': [0.98282142

In [8]:
df_train_scaling = df_train_post.copy()
for col in cont_names_2:
    df_train_scaling[col] = (df_train_scaling[col] - standardize_paras[col][0])/standardize_paras[col][1]
df_train_scaling.head(4)

,data_HR_activity_median,data_HR_activity_min,data_HR_activity_max,data_HR_activity_average,data_HR_activity_std,data_HR_activity_len,data_LEFT_ACC_MAG_median,data_LEFT_ACC_MAG_min,data_LEFT_ACC_MAG_max,data_LEFT_ACC_MAG_average,...,data_EOG_UD_by_activity_median,data_EOG_UD_by_activity_min,data_EOG_UD_by_activity_max,data_EOG_UD_by_activity_average,data_EOG_UD_by_activity_std,data_EOG_LR_by_activity_median,data_EOG_LR_by_activity_min,data_EOG_LR_by_activity_max,data_EOG_LR_by_activity_average,data_EOG_LR_by_activity_std
0,0.168533,0.320076,0.693515,0.170740,-0.041823,0.625804,-0.919668,0.254847,0.190162,-0.852501,...,-0.741745,0.119628,-0.216723,0.101100,-0.121325,0.139168,0.240213,-0.177826,0.362260,-0.380988
0,0.287748,0.775374,-0.058344,0.370003,-0.531763,0.625804,-0.899871,-0.290489,-0.337649,-0.810834,...,-0.767471,0.061925,-0.183696,1.072093,0.053807,0.409374,0.240437,-0.265232,0.455269,-0.381526
0,-1.443579,-1.466624,-1.750027,-1.533298,-0.313644,0.625804,-1.100996,-1.026626,-1.234384,-1.036088,...,-0.763299,0.783381,-0.513808,-0.115791,-0.503417,0.022372,0.022210,-0.038203,0.210303,0.056476
0,-1.122963,-0.720971,-1.464418,-1.153714,-0.719503,0.625804,-1.146419,-1.086029,-0.931140,-1.038276,...,-0.723182,0.697118,-0.457170,0.490562,-0.594354,-0.002105,-0.069446,0.115556,-0.425151,-0.053252


In [9]:
df_train_scaling['image_path'] = df_train['image_path']
df_train_scaling['label'] = df_train['label']

In [10]:
df_train_scaling

,data_HR_activity_median,data_HR_activity_min,data_HR_activity_max,data_HR_activity_average,data_HR_activity_std,data_HR_activity_len,data_LEFT_ACC_MAG_median,data_LEFT_ACC_MAG_min,data_LEFT_ACC_MAG_max,data_LEFT_ACC_MAG_average,...,data_EOG_UD_by_activity_min,data_EOG_UD_by_activity_max,data_EOG_UD_by_activity_average,data_EOG_UD_by_activity_std,data_EOG_LR_by_activity_median,data_EOG_LR_by_activity_min,data_EOG_LR_by_activity_max,data_EOG_LR_by_activity_average,data_EOG_LR_by_activity_std,label
0,0.168533,0.320076,0.693515,0.170740,-0.041823,0.625804,-0.919668,0.254847,0.190162,-0.852501,...,0.119628,-0.216723,0.101100,-0.121325,0.139168,0.240213,-0.177826,0.362260,-0.380988,1
0,0.287748,0.775374,-0.058344,0.370003,-0.531763,0.625804,-0.899871,-0.290489,-0.337649,-0.810834,...,0.061925,-0.183696,1.072093,0.053807,0.409374,0.240437,-0.265232,0.455269,-0.381526,1
0,-1.443579,-1.466624,-1.750027,-1.533298,-0.313644,0.625804,-1.100996,-1.026626,-1.234384,-1.036088,...,0.783381,-0.513808,-0.115791,-0.503417,0.022372,0.022210,-0.038203,0.210303,0.056476,1
0,-1.122963,-0.720971,-1.464418,-1.153714,-0.719503,0.625804,-1.146419,-1.086029,-0.931140,-1.038276,...,0.697118,-0.457170,0.490562,-0.594354,-0.002105,-0.069446,0.115556,-0.425151,-0.053252,1
0,-0.861720,-0.802992,-1.364205,-0.924028,-0.749108,0.625804,0.242324,0.025694,-0.841651,-0.034196,...,-0.376608,0.302816,-0.129109,1.131341,0.173097,0.094898,-0.115108,-0.015539,-0.065460,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,1.536575,1.150514,1.207945,1.600793,-0.258795,-1.592236,-0.560149,0.344601,0.677396,0.216965,...,0.096992,-0.316622,0.489574,0.232590,0.270706,0.185251,-0.090177,1.047977,0.054874,20
19,0.938846,1.282707,0.693515,0.899897,-0.590087,0.625804,-0.750199,0.645126,0.846592,0.019635,...,0.283924,-0.010588,-0.829440,0.263045,0.591525,0.146766,-0.162351,-0.110939,0.142913,20
19,1.081273,1.150514,1.030555,1.150357,-0.009890,-1.592236,-0.658615,0.386064,1.203071,0.330224,...,0.350303,0.655258,1.103702,0.000436,0.443035,0.088674,-0.170776,1.188128,0.076351,20
19,0.666745,0.008113,0.378219,0.507914,0.363736,-1.592236,0.267265,-0.590945,1.001614,-0.029348,...,-0.209558,0.167996,-4.998999,0.730619,-0.347505,0.037777,0.079822,2.029693,0.510928,20


In [11]:
joblib.dump(standardize_paras, 'scaling_paras_only_csv.joblib')

['scaling_paras_only_csv.joblib']

In [12]:
joblib.dump(df_train_scaling, 'tabular_only_csv.joblib')

['tabular_only_csv.joblib']

In [13]:
len(df_train_scaling)

1830

In [40]:
a = np.asarray(df_train_scaling.iloc[0,:])

In [23]:
# df = df.sample(frac=1).reset_index(drop=True)

In [44]:
a[0:5]

array([0.23799943188791783, 0.37100589227417063, 0.7645140355876211,
       0.23709961279216732, -0.027723933208150944], dtype=object)

In [35]:
b = np.array([19-1])

In [37]:
b[0]

18

In [2]:
import joblib
import pandas as pd

a = joblib.load('tabular_csv_only.joblib')

In [3]:
a

,data_HR_activity_median,data_HR_activity_min,data_HR_activity_max,data_HR_activity_average,data_HR_activity_std,data_HR_activity_len,data_LEFT_ACC_MAG_median,data_LEFT_ACC_MAG_min,data_LEFT_ACC_MAG_max,data_LEFT_ACC_MAG_average,...,data_EOG_UD_by_activity_min,data_EOG_UD_by_activity_max,data_EOG_UD_by_activity_average,data_EOG_UD_by_activity_std,data_EOG_LR_by_activity_median,data_EOG_LR_by_activity_min,data_EOG_LR_by_activity_max,data_EOG_LR_by_activity_average,data_EOG_LR_by_activity_std,label
0,0.168533,0.320076,0.693515,0.170740,-0.041823,0.625804,-0.919668,0.254847,0.190162,-0.852501,...,0.119628,-0.216723,0.101100,-0.121325,0.139168,0.240213,-0.177826,0.362260,-0.380988,1
0,0.287748,0.775374,-0.058344,0.370003,-0.531763,0.625804,-0.899871,-0.290489,-0.337649,-0.810834,...,0.061925,-0.183696,1.072093,0.053807,0.409374,0.240437,-0.265232,0.455269,-0.381526,1
0,-1.443579,-1.466624,-1.750027,-1.533298,-0.313644,0.625804,-1.100996,-1.026626,-1.234384,-1.036088,...,0.783381,-0.513808,-0.115791,-0.503417,0.022372,0.022210,-0.038203,0.210303,0.056476,1
0,-1.122963,-0.720971,-1.464418,-1.153714,-0.719503,0.625804,-1.146419,-1.086029,-0.931140,-1.038276,...,0.697118,-0.457170,0.490562,-0.594354,-0.002105,-0.069446,0.115556,-0.425151,-0.053252,1
0,-0.861720,-0.802992,-1.364205,-0.924028,-0.749108,0.625804,0.242324,0.025694,-0.841651,-0.034196,...,-0.376608,0.302816,-0.129109,1.131341,0.173097,0.094898,-0.115108,-0.015539,-0.065460,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,1.536575,1.150514,1.207945,1.600793,-0.258795,-1.592236,-0.560149,0.344601,0.677396,0.216965,...,0.096992,-0.316622,0.489574,0.232590,0.270706,0.185251,-0.090177,1.047977,0.054874,1
19,0.938846,1.282707,0.693515,0.899897,-0.590087,0.625804,-0.750199,0.645126,0.846592,0.019635,...,0.283924,-0.010588,-0.829440,0.263045,0.591525,0.146766,-0.162351,-0.110939,0.142913,1
19,1.081273,1.150514,1.030555,1.150357,-0.009890,-1.592236,-0.658615,0.386064,1.203071,0.330224,...,0.350303,0.655258,1.103702,0.000436,0.443035,0.088674,-0.170776,1.188128,0.076351,1
19,0.666745,0.008113,0.378219,0.507914,0.363736,-1.592236,0.267265,-0.590945,1.001614,-0.029348,...,-0.209558,0.167996,-4.998999,0.730619,-0.347505,0.037777,0.079822,2.029693,0.510928,1


In [6]:
b = a.sample(frac=1)#.reset_index(drop=True)

In [7]:
b

,data_HR_activity_median,data_HR_activity_min,data_HR_activity_max,data_HR_activity_average,data_HR_activity_std,data_HR_activity_len,data_LEFT_ACC_MAG_median,data_LEFT_ACC_MAG_min,data_LEFT_ACC_MAG_max,data_LEFT_ACC_MAG_average,...,data_EOG_UD_by_activity_min,data_EOG_UD_by_activity_max,data_EOG_UD_by_activity_average,data_EOG_UD_by_activity_std,data_EOG_LR_by_activity_median,data_EOG_LR_by_activity_min,data_EOG_LR_by_activity_max,data_EOG_LR_by_activity_average,data_EOG_LR_by_activity_std,label
13,0.052630,-0.187334,-0.327667,0.005930,-0.033501,0.625804,-0.216853,2.024361,0.324971,-0.413370,...,0.430507,-0.527455,0.142943,-0.121855,-0.031386,0.243789,-0.253143,0.181361,-0.242150,1
2,-0.060098,0.109501,-0.327667,-0.088175,-0.248876,0.625804,0.670373,-0.860796,1.683793,0.265885,...,0.193638,-0.690799,-0.182285,-0.614964,-0.407539,0.283245,-0.280764,0.728536,-0.421597,1
3,-1.665544,-1.466624,0.378219,-1.656121,0.478210,0.625804,0.136898,0.436640,-0.722395,-0.126375,...,0.095278,-0.645141,0.126783,-0.870212,-0.127929,0.378845,-0.379244,-0.004219,-0.605016,1
18,0.666745,1.021994,0.082629,0.659680,-0.510968,0.625804,1.081638,-1.694028,1.668736,1.637145,...,-1.031950,0.905273,1.435240,3.284424,0.169944,0.008289,-0.058929,-0.712453,0.159359,1
16,0.052630,-0.090802,0.082629,-0.013617,0.000674,-1.592236,3.115953,-1.522676,2.686579,3.542451,...,0.848179,-0.565001,0.260757,-0.894346,-0.066521,-0.137679,-0.045284,0.174648,0.466749,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,-0.380483,-0.281568,-0.581376,-0.440804,-0.188918,-1.592236,0.181311,-1.529723,0.083260,-0.055012,...,-0.044928,-0.461471,1.577228,0.306754,0.441080,0.309775,-0.349299,0.356305,-0.549008,1
14,0.228141,-0.090802,0.228078,0.221925,0.324483,0.625804,-0.892802,-1.227204,-0.826745,-0.673741,...,-1.738059,1.049177,-0.002640,3.130233,0.247200,0.016248,-0.033926,0.076677,0.386816,1
3,-0.950925,-0.551270,-0.581376,-0.991511,-0.270837,-1.592236,-0.525024,0.752147,-0.341368,-0.572826,...,0.223485,-0.838664,0.252971,-1.258978,0.112099,0.384327,-0.398348,-0.156764,-0.635847,1
12,-0.861720,-1.532987,-0.456387,-0.934104,0.834774,0.625804,-0.660022,-1.060699,-0.884233,-0.166980,...,-1.128669,1.693866,-1.655248,1.535913,0.197215,0.053584,-0.079511,1.096173,-0.084045,1


In [9]:
c = b.iloc[0:10,:]
c

,data_HR_activity_median,data_HR_activity_min,data_HR_activity_max,data_HR_activity_average,data_HR_activity_std,data_HR_activity_len,data_LEFT_ACC_MAG_median,data_LEFT_ACC_MAG_min,data_LEFT_ACC_MAG_max,data_LEFT_ACC_MAG_average,...,data_EOG_UD_by_activity_min,data_EOG_UD_by_activity_max,data_EOG_UD_by_activity_average,data_EOG_UD_by_activity_std,data_EOG_LR_by_activity_median,data_EOG_LR_by_activity_min,data_EOG_LR_by_activity_max,data_EOG_LR_by_activity_average,data_EOG_LR_by_activity_std,label
13,0.052630,-0.187334,-0.327667,0.005930,-0.033501,0.625804,-0.216853,2.024361,0.324971,-0.413370,...,0.430507,-0.527455,0.142943,-0.121855,-0.031386,0.243789,-0.253143,0.181361,-0.242150,1
2,-0.060098,0.109501,-0.327667,-0.088175,-0.248876,0.625804,0.670373,-0.860796,1.683793,0.265885,...,0.193638,-0.690799,-0.182285,-0.614964,-0.407539,0.283245,-0.280764,0.728536,-0.421597,1
3,-1.665544,-1.466624,0.378219,-1.656121,0.478210,0.625804,0.136898,0.436640,-0.722395,-0.126375,...,0.095278,-0.645141,0.126783,-0.870212,-0.127929,0.378845,-0.379244,-0.004219,-0.605016,1
18,0.666745,1.021994,0.082629,0.659680,-0.510968,0.625804,1.081638,-1.694028,1.668736,1.637145,...,-1.031950,0.905273,1.435240,3.284424,0.169944,0.008289,-0.058929,-0.712453,0.159359,1
16,0.052630,-0.090802,0.082629,-0.013617,0.000674,-1.592236,3.115953,-1.522676,2.686579,3.542451,...,0.848179,-0.565001,0.260757,-0.894346,-0.066521,-0.137679,-0.045284,0.174648,0.466749,1
2,0.536697,1.021994,-0.058344,0.528180,-0.867118,-1.592236,0.355858,-0.475917,-0.001048,0.040080,...,0.044408,-0.221213,0.637208,0.709336,-0.066561,0.291286,-0.299471,1.265364,-0.432101,1
15,0.168533,0.541735,0.533283,0.185871,-0.282218,0.625804,-0.578423,1.441387,-0.498201,-0.606452,...,1.215918,-0.697184,-0.027024,-1.310873,-0.002676,0.358630,-0.323768,-0.044175,-0.537340,1
16,0.287748,0.109501,0.378219,0.304808,0.034180,0.625804,-0.216853,0.613143,0.291353,-0.266968,...,-0.060920,-0.318293,0.105443,-0.055190,1.133709,0.174586,-0.210055,1.020610,-0.220282,1
3,0.800734,0.213456,0.228078,0.808193,-0.237363,0.625804,-0.654391,1.790337,-1.173042,-0.645942,...,0.434488,-0.691898,-0.163013,-1.083612,-0.235550,0.373916,-0.372130,0.109767,-0.567940,1
12,0.052630,-0.551270,0.859179,0.063261,1.023521,0.625804,-0.466063,-0.950222,0.262873,0.329941,...,-0.303184,0.568390,-6.048622,1.040085,0.327918,0.160777,-0.183505,-1.703503,-0.264459,1


In [11]:
len(c.columns)

103

In [2]:
import joblib
from dataloader import MyDatasetGenerator
from torch.utils.data import DataLoader

a = joblib.load('tabular_csv_only.joblib')

In [3]:
a = a.sample(frac=1)
train_portion = 0.8
train_numb = int(train_portion * len(a))
train_df = a.iloc[0:train_numb, :]
val_df = a.iloc[train_numb:, :]

In [5]:
a

,data_HR_activity_median,data_HR_activity_min,data_HR_activity_max,data_HR_activity_average,data_HR_activity_std,data_HR_activity_len,data_LEFT_ACC_MAG_median,data_LEFT_ACC_MAG_min,data_LEFT_ACC_MAG_max,data_LEFT_ACC_MAG_average,...,data_EOG_UD_by_activity_min,data_EOG_UD_by_activity_max,data_EOG_UD_by_activity_average,data_EOG_UD_by_activity_std,data_EOG_LR_by_activity_median,data_EOG_LR_by_activity_min,data_EOG_LR_by_activity_max,data_EOG_LR_by_activity_average,data_EOG_LR_by_activity_std,label
2,-1.665544,-1.466624,-1.750027,-1.718097,-0.256425,0.625804,-0.922497,-1.378337,-0.301636,-0.799305,...,1.004830,-0.720828,-0.764619,-1.093923,-0.427552,0.047820,0.160400,-2.498169,0.179968,1
17,0.800734,0.896995,0.378219,0.820589,-0.342420,-1.592236,-1.181931,-0.452192,0.890681,-0.348959,...,0.585481,-0.578588,-0.226460,-0.586665,-1.296535,-0.027282,0.220675,-0.926893,0.332659,1
9,0.168533,0.656997,-0.195046,0.215289,-0.566391,0.625804,-0.191712,1.397471,-0.737602,-0.384945,...,0.641274,-0.580365,0.958238,-1.083619,-0.767762,0.269602,-0.264048,0.234964,-0.240075,1
11,-1.805565,-1.532987,-1.562062,-1.775257,-0.031648,0.625804,0.297734,0.697024,-1.095970,-0.089327,...,0.124101,-0.435033,-0.973331,-0.255841,-0.933336,0.201759,-0.262422,-0.576637,-0.371224,1
14,0.410419,0.429466,-0.058344,0.332348,-0.221139,0.625804,0.444315,-1.290409,-0.621883,0.030756,...,0.583769,-0.386335,-0.115743,-0.208842,1.554931,-9.794801,9.597018,0.304375,10.293667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,0.168533,0.541735,0.533283,0.185871,-0.282218,0.625804,-0.578423,1.441387,-0.498201,-0.606452,...,1.215918,-0.697184,-0.027024,-1.310873,-0.002676,0.358630,-0.323768,-0.044175,-0.537340,1
17,3.217687,2.493758,2.887716,3.172085,1.069445,0.625804,-1.332752,-0.026348,1.262612,-0.192022,...,0.066241,0.054409,-0.101747,0.535991,3.436071,-0.136934,0.020098,-0.630198,0.851492,1
11,-0.861720,-0.883212,-1.155652,-0.960788,-0.553782,0.625804,-0.091259,0.454182,-1.012463,-0.284195,...,-0.563150,-0.588854,0.152758,-0.332040,0.002461,0.201279,-0.313239,-0.395437,-0.392682,1
8,0.869790,1.282707,0.378219,0.864461,-0.643582,0.625804,0.400103,-0.906753,-0.492036,0.135210,...,0.438212,-0.508359,0.235980,-0.978711,-0.068834,0.202185,-0.251667,0.010019,-0.445807,1
